In [15]:
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModel
import torch
import subprocess
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection
import requests
import json
from pprint import pp, pprint
import os
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from time import sleep

## 讀檔案

In [ ]:
df = pd.read_csv(
    r"C:\Users\student\期末專題\prompt提取技能\oId_data\104_programmers_jobs.csv"
)



df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15748 entries, 0 to 15747
Data columns (total 46 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  15748 non-null  object 
 1   更新日期    15748 non-null  object 
 2   查詢職類    15748 non-null  int64  
 3   查詢關鍵字   15748 non-null  object 
 4   職務類別    15748 non-null  object 
 5   職缺名稱    15748 non-null  object 
 6   公司名稱    15748 non-null  object 
 7   公司連結    15748 non-null  object 
 8   產業別     15748 non-null  object 
 9   上班地點    15748 non-null  object 
 10  地區      14577 non-null  object 
 11  城市      15748 non-null  object 
 12  國家      15748 non-null  object 
 13  薪資      15748 non-null  object 
 14  薪資下限    15748 non-null  int64  
 15  薪資上限    15748 non-null  int64  
 16  職缺描述    15733 non-null  object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  15748 non-null  object 
 19  學歷要求    15748 non-null  object 
 20  科系要求    8534 non-null   object 
 21  擅長工具    11180 non-null  object 
 22

# GPT-API

In [12]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os
from openai import OpenAI

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [13]:
df = df.head(100).copy()
df["技能原文"] = (
    df["職缺描述"].astype(str)
    + "\n"
    + df["擅長工具"].astype(str)
    + "\n"
    + df["工作技能"].astype(str)
)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 47 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  100 non-null    object 
 1   更新日期    100 non-null    object 
 2   查詢職類    100 non-null    int64  
 3   查詢關鍵字   100 non-null    object 
 4   職務類別    100 non-null    object 
 5   職缺名稱    100 non-null    object 
 6   公司名稱    100 non-null    object 
 7   公司連結    100 non-null    object 
 8   產業別     100 non-null    object 
 9   上班地點    100 non-null    object 
 10  地區      94 non-null     object 
 11  城市      100 non-null    object 
 12  國家      100 non-null    object 
 13  薪資      100 non-null    object 
 14  薪資下限    100 non-null    int64  
 15  薪資上限    100 non-null    int64  
 16  職缺描述    100 non-null    object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  100 non-null    object 
 19  學歷要求    100 non-null    object 
 20  科系要求    41 non-null     object 
 21  擅長工具    66 non-null     object 
 22  工作技

In [ ]:
# ✅ 定義 GPT 呼叫函數



# ✅ 建立 GPT 呼叫函數（可多線使用）
def extract_skills_gpt(text_and_id, model="gpt-4.1-nano"):
    idx, text = text_and_id

    prompt = f"""
你現在是一位資深的軟體工程師，
熟悉市面上全部跟軟體相關的技能。
也擅長讀懂職缺JD裡面的內容，
並分析出職缺所需要的技能。
請從下列職缺內容中列出會學到的技能或工具，並用分號分隔。
並在工具或技能名稱後面附上這職缺會用到這個技能的哪些部分。

請遵守以下規則：
1. 不要包含任何軟實力（如：溝通能力、團隊合作）
2. 使用繁體中文及英文回覆
3. 回復範例如下：
資料科學（Data Science）:學習資料收集、探索性分析 (EDA) 與特徵工程在 AI 專案中的角色;
人工神經網路（Artificial Neural Networks）:掌握神經元結構、前向傳播與反向傳播的基本原理;
...

以下是職缺內容：
{text}
"""
    try:
        sleep(0.1)
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=512,
        )
        return idx, response.choices[0].message.content.strip()
    except Exception as e:
        return idx, f"錯誤：{e}"


# ✅ 主程式邏輯（推薦放在 main 區塊）
if __name__ == "__main__":
    # 讀入資料（請根據你實際路徑修改）
    texts = df["技能原文"].astype(str).tolist()
    data_with_id = list(enumerate(texts))

    # 儲存結果用的 dict
    results_dict = {}

    # 建立 ThreadPoolExecutor + tqdm 追蹤進度
    with ThreadPoolExecutor(max_workers=50) as executor:  # 可調整執行緒數
        futures = [executor.submit(extract_skills_gpt, item) for item in data_with_id]
        for future in tqdm(as_completed(futures), total=len(futures), desc="處理中"):
            idx, result = future.result()
            results_dict[idx] = result

    # 回填結果，維持原順序
    results = [results_dict[i] for i in range(len(texts))]
    df["技能"] = results

    # 匯出結果
    df.to_csv("output_104_skills_test_1040723.csv", index=False, encoding="utf-8-sig")

處理中: 100%|██████████| 100/100 [00:10<00:00,  9.97it/s]


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 48 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   job_id  5 non-null      object 
 1   更新日期    5 non-null      object 
 2   查詢職類    5 non-null      int64  
 3   查詢關鍵字   5 non-null      object 
 4   職務類別    5 non-null      object 
 5   職缺名稱    5 non-null      object 
 6   公司名稱    5 non-null      object 
 7   公司連結    5 non-null      object 
 8   產業別     5 non-null      object 
 9   上班地點    5 non-null      object 
 10  地區      5 non-null      object 
 11  城市      5 non-null      object 
 12  國家      5 non-null      object 
 13  薪資      5 non-null      object 
 14  薪資下限    5 non-null      int64  
 15  薪資上限    5 non-null      int64  
 16  職缺描述    5 non-null      object 
 17  職務需求    0 non-null      float64
 18  工作經歷要求  5 non-null      object 
 19  學歷要求    5 non-null      object 
 20  科系要求    1 non-null      object 
 21  擅長工具    4 non-null      object 
 22  工作技能  

In [ ]:
# ✅ 擷取你要檢查的欄位
df_test = df[["技能", "技能原文"]]
pp(df_test["技能"].iloc[1])
# print(df_test["技能"].iloc[0])

('前端開發（Front-end Development）:掌握 HTML、CSS、JavaScript、TypeScript 的基本技能與應用;  \n'
 '前端框架（Front-end Frameworks）:熟悉 React、Next.js、Vue、Nuxt.js 的開發經驗;  \n'
 '版本控制（Version Control）:使用 Git Flow 開發流程進行版本管理與協作;  \n'
 '3D Web 技術（WebXR, WebGL, Three.js）:了解 WebXR 技術、WebGL 及 Three.js '
 '相關開發經驗，用於沉浸式體驗與 3D 場景建構;  \n'
 '效能優化（Performance Optimization）:提供網站效能優化建議，提升使用者體驗;  \n'
 '軟體工程（Software Engineering）:系統架構規劃、模組化系統設計、結構化程式設計與系統整合分析;  \n'
 '後端技術（Backend Technologies）:Node.js 相關開發經驗，用於後端服務與 API 整合;  \n'
 '雲端服務（Cloud Services）:AWS 使用經驗，用於部署與擴展應用;  \n'
 '資料庫應用（Database Applications）:資料庫軟體應用，用於資料存取與管理;')


In [ ]:
df_split = df["技能"].str.split(";")

# 找出最多的技能數量，決定要建立幾欄
max_len = df_split.map(len).max()

# 拆分成多欄，並命名為 技能1、技能2、技能3...
df_skills = pd.DataFrame(
    df_split.tolist(), columns=[f"技能{i+1}" for i in range(max_len)]
)

# 把這些新欄位合併回原本的 dataframe（df_test）
df_result = pd.concat([df, df_skills], axis=1)

# 顯示結果
df_result.head()

,job_id,更新日期,查詢職類,查詢關鍵字,職務類別,職缺名稱,公司名稱,公司連結,產業別,上班地點,...,技能11,技能12,技能13,技能14,技能15,技能16,技能17,技能18,技能19,技能20
0,8f54d,2025/06/30,2007001001,TypeScript,軟體工程師、前端工程師、軟體專案主管,[R&D] Frontend Engineer 前端工程師,iStaging愛實境_愛實境股份有限公司,https://www.104.com.tw/company/bjdang8,其它軟體及網路相關業,台北市松山區南京東路4段2號2樓（小巨蛋南區入口處）,...,\nWebXR（WebXR）:開發虛擬實境（VR）、擴增實境（AR）與混合實境（MR）體驗,\nWebGL（WebGL）:3D圖形渲染與互動效果實作,\nThree.js（Three.js）:基於WebGL的3D圖形庫，用於建立3D場景與動畫,\nNode.js（Node.js）:後端服務開發、API串接與伺服器端邏輯,\nAWS（Amazon Web Services）:雲端服務部署、資源管理與網站效能優化,\n網站效能優化（Web Performance Optimization）:提升網站載...,\n系統架構規劃（System Architecture Design）:設計系統整體架...,\n資料庫軟體應用（Database Management）:資料存取、查詢與資料庫設計,\n模組化系統設計（Modular System Design）:建立可擴展與維護性高的...,\n結構化程式設計（Structured Programming）:提升程式碼清晰度與可讀
1,7p2cn,2025/06/30,2007001001,TypeScript,軟體工程師、前端工程師、軟體專案主管,[R&D] Senior Frontend Engineer 資深前端工程師,iStaging愛實境_愛實境股份有限公司,https://www.104.com.tw/company/bjdang8,其它軟體及網路相關業,台北市松山區南京東路4段2號2樓（小巨蛋南區入口處）,...,,None,None,None,None,None,None,None,None,None
2,8cbyv,2025/07/14,2007001001,TypeScript,後端工程師、軟體工程師、軟體專案主管,後端工程主管,香港商蒙卓有限公司台灣分公司,https://www.104.com.tw/company/1a2x6bmedx,網際網路相關業,台北市松山區南京東路五段99號5樓,...,進行故障排除與性能優化 \n軟體工程系統開發（Software System Devel...,進行系統分析與設計 \n軟體程式設計（Software Programming）: 編寫...,進行模組化與重構 \n網路程式設計（Network Programming）: 開發與優...,進行網路協定的應用與調試 \n資料庫程式設計（Database Programming）...,None,None,None,None,None,None
3,8b902,2025/07/09,2007001001,TypeScript,軟體工程師、軟體專案主管、演算法工程師,Senior Software Development Lead,Taiwan AI Labs_雅婷智慧股份有限公司,https://www.104.com.tw/company/1a2x6bm0oj,其它軟體及網路相關業,台北市大同區承德路1段70號15樓,...,,None,None,None,None,None,None,None,None,None
4,8bz2q,2025/07/14,2007001001,TypeScript,前端工程師、軟體工程師、軟體專案主管,【擴編】資深前端工程師 Senior Frontend Developer,思量科技有限公司,https://www.104.com.tw/company/1a2x6bmxjc,電腦軟體服務業,台北市中正區忠孝東路二段26號2樓,...,\n團隊技術指導（Technical Leadership）:提供技術指導與團隊協作支持,\n系統整合（System Integration）:與後端系統進行整合，確保前後端協同運作,,None,None,None,None,None,None,None


In [ ]:
# 篩選出要保留的欄位（技能開頭且後面是數字的）
keep_columns = [
    col for col in df_result.columns if col.startswith("技能") and col[2:].isdigit()
]

# 篩選出要刪除的欄位（技能開頭但後面不是純數字的）
drop_columns = [
    col for col in df_result.columns if col.startswith("技能") and not col[2:].isdigit()
]

# 刪除這些不要的欄位
df_result.drop(columns=drop_columns, inplace=True)

# 最後確認留下來的
df_result

In [33]:
df_result.to_csv("104_skills_1140721.csv", index=False, encoding="utf-8-sig")